In [2]:
from transformers import pipeline
from dotenv import load_dotenv
import pandas as pd 

load_dotenv()


False

In [3]:
from datasets import load_dataset

ds = load_dataset("Aarif1430/english-to-hindi")

In [73]:
train=pd.DataFrame(ds["train"])

In [84]:
train.iloc[0,0]

"However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles"

In [75]:
train['hindi_sentence']

0         आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...
1         और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2         जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...
3           आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
4         8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...
                                ...                        
127700    आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127701                      और अपने गालों में डाल लेते हैं।
127702    जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द...
127703    Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .
127704    हाल ही में उन्हें सरकारी ठेका मिला है करीब सौ ...
Name: hindi_sentence, Length: 127705, dtype: str

In [76]:
train.describe()

,english_sentence,hindi_sentence
count,127705,127705
unique,124417,97762
top,(Laughter),(हँसी)
freq,555,212


In [ ]:
from transformers import AutoTokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Load the model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# We tell the model to expect 2 categories (e.g., 'Positive', 'Negative')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 2. Prepare an input
text = "This tutorial is incredibly detailed!"
inputs = tokenizer(text, return_tensors="pt")

# 3. Run the model
# Internally: 
#   - Inputs go into the DistilBERT Body.
#   - Body outputs a vector (hidden state) of size 768.
#   - The 'Head' takes that 768 vector and projects it to 2 numbers (logits).
outputs = model(**inputs)

# 4. Access the Logits
# Logits are raw scores from the final layer BEFORE the softmax.
# They are NOT probabilities yet (they can be negative or > 1).
logits = outputs.logits

# 5. Convert Logits to Probabilities
# torch.nn.functional.softmax turns scores into 0-1 range that sums to 1.
predictions = torch.nn.functional.softmax(logits, dim=-1)

print(f"Raw Logits: {logits}")
print(f"Probabilities: {predictions}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Raw Logits: tensor([[-0.0721, -0.0458]], grad_fn=<AddmmBackward0>)
Probabilities: tensor([[0.4934, 0.5066]], grad_fn=<SoftmaxBackward0>)


In [10]:
tokenizer.batch_decode(logits, skip_special_tokens=True)

TypeError: argument 'ids': 'float' object cannot be interpreted as an integer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load a generative model (decoder-only)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Tokenize the prompt
prompt = "The secret to learning AI is"
# .to("cpu") or .to("cuda") ensures the data is where the model is.
inputs = tokenizer(prompt, return_tensors="pt")

# 3. The .generate() method - The engine of creation
# - max_new_tokens: How many words to add (ignores prompt length).
# - do_sample=True: Enables creative randomness (not just the top word).
# - temperature=0.7: 'Cooler' (low) is more factual; 'Hotter' (high) is more random.
# - top_p=0.9: 'Nucleus sampling' - only looks at the top 90% of likely words.
output_tokens = model.generate(
    **inputs, 
    max_new_tokens=30, 
    do_sample=True, 
    temperature=0.7, 
    top_p=0.9
)

# 4. Decode the numbers back to words
# batch_decode handles a list of sequences; [0] takes the first (and only) result.
# skip_special_tokens=True removes technical markers like <|endoftext|>.
decoded_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

print(f"Generated: {decoded_text}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: The secret to learning AI is to build it from the ground up. That means that, even if you can make an AI that does something you'd never dream of doing, you


In [3]:
import torch

In [7]:
tensor=torch.tensor(data=[[1,2,3],[4,5,6]])

In [ ]:
tensor.ndim

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset # A sister library for efficient data loading

# 1. THE DATA (The "Fuel")
# We load a simple dataset. Internally, this is handled as a memory-mapped 
# file so it doesn't crash your RAM even if the dataset is 100GB.
dataset = load_dataset("imdb")

# 2. THE TOKENIZER (The "Translator")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Line-by-line function to process the data:
def tokenize_function(examples):
    # This adds 'input_ids' and 'attention_mask' to the dataset.
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 3. THE MODEL (The "Student")
# We load the body (DistilBERT) and add a fresh, random "Head" for 2-class classification.
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 4. THE ARGUMENTS (The "Lesson Plan")
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",  # Run a test after every pass through the data
    learning_rate=2e-5,           # How big of a 'step' the model takes when learning
    weight_decay=0.01,            # A penalty to prevent the model from 'over-memorizing'
    num_train_epochs=3,           # Total times to read the whole book
)

# 5. THE TRAINER (The "Teacher")
# This object coordinates the Model, the Args, and the Data.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# 6. THE EXECUTION
# This one line replaces ~100 lines of raw PyTorch training loop code.
trainer.train()

Error importing huggingface_hub.hf_file_system: cannot import name 'http_stream_backoff' from 'huggingface_hub.utils' (/Users/divyyadav/Desktop/transformer/.venv/lib/python3.12/site-packages/huggingface_hub/utils/__init__.py)


ImportError: cannot import name 'http_stream_backoff' from 'huggingface_hub.utils' (/Users/divyyadav/Desktop/transformer/.venv/lib/python3.12/site-packages/huggingface_hub/utils/__init__.py)

In [1]:
arr=[1,2,3,4,5]

b=0

for i in arr:
    b+=i

In [53]:
class SortMedian:
    def __init__(self, arr, arr2):
        self.arr = arr
        self.arr2 = arr2
        self.final_array = self.arr + self.arr2

    def combined_median(self, sort: bool = True):
        try:
            if sort:
                sorted_array = sorted(self.final_array)
            else:
                sorted_array = self.final_array

            length = len(sorted_array)

            if length % 2 == 0:
                x = sorted_array[length//2]
                y = sorted_array[length//2 - 1]
                return (x + y) / 2
            else:
                return sorted_array[length//2]
        except Exception as e:
            print(f"Error: {e}")
            return None

In [57]:
medi=SortMedian([1,2,3,4,5], [6,7,8,9,10,9])

In [58]:
medi.final_array

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9]

In [59]:
medi.combined_median()

6

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

from datasets import load_dataset
from tokenizers.models import WordLevel
from tokenizers import Tokenizer
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

In [ ]:
ds=load_dataset("Aarif1430/english-to-hindi",split='train')


In [25]:
def get_all_sentences(ds,lang):
    for items in ds:
        yield items

In [20]:
def get_or_build_tokenizer(config,ds,lang):
    #define the tokenizer path
    path=Path(config["tokenizer_path"].format(lang))

    if not path.exists():
        #build the tokenizer
        tokenizer=Tokenizer(WordLevel(unk_token='[UNK]'))
        #Pre tokenizer
        tokenizer.pre_tokenizer=Whitespace()
        #Trainer
        trainer=WordLevelTrainer(min_frequency=2,special_tokens=['[PAD]','[UNK]','[SOS]','[EOS]'])

        #train the tokenizer using the trainer
        tokenizer.train_from_iterator(get_all_sentences(ds),trainer=trainer)

        #save the tokenizer
        tokenizer.save(path)

    else:
        tokenizer=Tokenizer.from_file(path)

    return tokenizer